In [ ]:
!pip install pymongo pyngrok
!pip install ngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 54.7 MB/s eta 0:00:00


In [8]:
import hashlib
import pickle
import pandas as pd
from flask import Flask, render_template, request, redirect, url_for, session, flash, jsonify
from pymongo import MongoClient
from pyngrok import ngrok
from urllib.parse import quote_plus
import joblib

ngrok.set_auth_token("2os4BpqJxJ2zMlPIKnogyPJesjm_7PXoEfByyvcRCQ6RUaz87")
app = Flask(__name__)
app.secret_key = 'MANJU'  # Replace with a secure key

# MongoDB Atlas connection
username = "manjunath"  # Replace with your MongoDB Atlas username
password = "manju@2005"  # Replace with your MongoDB Atlas password
cluster_url = "cluster0.9gafb.mongodb.net"  # Replace with your MongoDB Atlas cluster URL
db_name = "Project0"  # Database name

# URL-encode the username and password to comply with RFC 3986
encoded_username = quote_plus(username)
encoded_password = quote_plus(password)

# Create connection string and client for MongoDB Atlas
uri = f"mongodb+srv://{encoded_username}:{encoded_password}@{cluster_url}/{db_name}?retryWrites=true&w=majority"
client = MongoClient(uri)  # Initialize MongoClient
db = client.get_database(db_name)  # Replace with your database name
users_collection = db.cancer_data  # Your collection name for users

# Load your pre-trained model
model = joblib.load(open('iris_species_model.joblib', 'rb'))  # Ensure this path points to your trained Iris model
feature_names = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']

@app.route('/')
def index():
    return redirect(url_for('login'))  # Redirect to login page on visiting the root URL

@app.route('/login', methods=['GET'])
def login():
    return render_template('login.html')  # Render the login page

@app.route('/signup', methods=['POST'])
def signup():
    username = request.form['username']
    password = request.form['password']
    confirm_password = request.form['confirm_password']

    if password != confirm_password:
        flash("Passwords do not match!")
        return redirect(url_for('login'))

    # Hash the password
    hashed_password = hashlib.sha256(password.encode()).hexdigest()

    # Check if the username already exists
    existing_user = users_collection.find_one({"username": username})
    if existing_user:
        flash("Username already exists. Please choose another one.")
        return redirect(url_for('login'))

    # Register the new user in the database
    users_collection.insert_one({"username": username, "password": hashed_password})
    flash('Signup successful! Please sign in to continue.')
    return redirect(url_for('login'))

@app.route('/signin', methods=['POST'])
def signin():
    username = request.form['username']
    password = request.form['password']

    # Hash the password
    hashed_password = hashlib.sha256(password.encode()).hexdigest()

    # Check user credentials in MongoDB
    user = users_collection.find_one({"username": username, "password": hashed_password})

    if user:
        # Store user session data
        session['username'] = username
        flash("Signin successful!")
        return redirect(url_for('ProjectHomepage'))  # Redirect to homepage
    else:
        flash("Invalid credentials! Please try again.")
        return redirect(url_for('login'))  # Redirect back to login page if authentication fails


@app.route('/ProjectHomepage', methods=['GET', 'POST'])
def ProjectHomepage():
    if 'username' not in session:
        return jsonify({"error": "Please login first."}), 401

    prediction = None  # Default value for prediction

    if request.method == 'POST':
        try:
            # Collect user inputs
            input_values = [
                float(request.form.get('SepalLengthCm', 0)),
                float(request.form.get('SepalWidthCm', 0)),
                float(request.form.get('PetalLengthCm', 0)),
                float(request.form.get('PetalWidthCm', 0))
            ]

            # Validate inputs
            if not all(input_values):
                return jsonify({"error": "All fields are required and must be valid numbers."}), 400

            # Prepare input data for prediction
            input_df = pd.DataFrame([input_values], columns=feature_names)

            # Predict using the model
            prediction_code = model.predict(input_df)[0]

            # Map numeric prediction to Iris species names
            species_map = {0: 'Setosa', 1: 'Versicolor', 2: 'Virginica'}
            prediction = species_map.get(prediction_code, "Unknown")
            return jsonify({"prediction": prediction}), 200
        except Exception as e:
           return jsonify({"error": f"Error during prediction: {e}"}), 500

    return render_template('ProjectHomepage.html', prediction=prediction)



if __name__ == "__main__":
    port = 5000
    public_url = ngrok.connect(port)
    print(f'Ngrok URL: {public_url}')
    app.run(port=port, debug=False)


Ngrok URL: NgrokTunnel: "https://b49f-34-34-67-32.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [22/Nov/2024 07:32:10] "GET / HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [22/Nov/2024 07:32:10] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Nov/2024 07:32:11] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [22/Nov/2024 07:32:19] "POST /signin HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [22/Nov/2024 07:32:19] "GET /ProjectHomepage HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Nov/2024 07:32:42] "POST /ProjectHomepage HTTP/1.1" 200 -
